In [1]:
import pandas as pd
import scipy
from scipy import stats
import numpy as np
import datetime
import matplotlib as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
import sklearn
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import scipy 
import arcgis

In [2]:


# read data
kc_columns = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'condition', 'grade', 'sqft_above',
              'sqft_basement', 'yr_built']

kc_dtypes = {'id': int, 'date' : str,  'price': float, 'bedrooms' : int, 'bathrooms' : float, 'sqft_living': int, 'sqft_lot': int,
             'floors': float, 'waterfront': float, 'view' : float, 'condition': float, 'grade': int, 'sqft_above': int,
             'yr_built': int, 'yr_renovated': float, 'zipcode': float, 'lat': float, 'long': float}

kc_data = pd.read_csv('data/kc_house_data.csv', dtype = kc_dtypes, parse_dates = ['date'])

kc_data['sqft_basement'] = kc_data['sqft_basement'].replace({'?': 0})
kc_data['sqft_basement'] = kc_data['sqft_basement'].astype(dtype=float, errors='ignore')

In [3]:
#drop unnessecary columns and fix data
kc_data = kc_data.drop('sqft_living15', 1).drop('sqft_lot15', 1)

#look for outliers, in bedrooms, we can clearly see a single outlier, for other columns, filtering by z score will be easiest
kc_data[kc_data['bedrooms'] == 33]
# wouldn't be realistic for a house with 33 bedrooms to only have a sqft_living of 1620 and only 1 3/4 bathrooms so it looks like a typo
# will adjust to 3
kc_data[kc_data['bedrooms'] == 33] = kc_data[kc_data['bedrooms'] == 33].replace(33,3)

In [4]:
#setting waterfront NaN values equal to the ratio of waterfront/non-waterfront properties, will want to try and narrow by zipcode
#filling NaN with easily seperatable/changable values helpful
kc_data['waterfront'] = kc_data['waterfront'].fillna(146/19221)
kc_data['view'] = kc_data['view'].fillna((957 + (508*2) + (330*3) + (317*4))/21534)
kc_data['yr_renovated'] = kc_data['waterfront'].fillna(0)

In [5]:
#Convert to integer for whole number year
kc_data['yr_renovated'] = kc_data['yr_renovated'].astype('int')

In [6]:
def norm_feat(column):
    """Give column from data frame, returns column z scores"""
    return (column - column.mean())/column.std()
for column in kc_columns:
    kc_data[column] = norm_feat(kc_data[column])

In [7]:
for column in kc_columns:
    kc_data[column] = kc_data.loc[kc_data[column] < 3]

In [8]:
for col in kc_columns:
    col_zscore = col + '_zscore'
    kc_data[col_zscore] = (kc_data[col] - kc_data[col].mean())/kc_data[col].std(ddof=0)

In [9]:
for col in kc_columns:
    kc_data.drop(col, 1, inplace=True)

In [10]:
kc_data.price_zscore

0        -1.21876
1         1.78023
2         1.12459
3        -1.50958
4         1.64222
           ...   
21592     0.22523
21593    -1.66214
21594     1.28106
21595    0.248947
21596     1.28106
Name: price_zscore, Length: 21597, dtype: object

In [12]:
kc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    21597 non-null  int32         
 1   date                  21597 non-null  datetime64[ns]
 2   waterfront            21597 non-null  float64       
 3   view                  21597 non-null  float64       
 4   yr_renovated          21597 non-null  int32         
 5   zipcode               21597 non-null  float64       
 6   lat                   21597 non-null  float64       
 7   long                  21597 non-null  float64       
 8   price_zscore          21191 non-null  object        
 9   bedrooms_zscore       21536 non-null  object        
 10  bathrooms_zscore      21410 non-null  object        
 11  sqft_living_zscore    21349 non-null  object        
 12  sqft_lot_zscore       21251 non-null  object        
 13  floors_zscore   

In [11]:
kc_data.to_csv('data/Normalized_Dataset.csv', index=False)